## KomaMRI im Optimized Shaped Pulse

In [ ]:
using KomaMRI, MAT, Plots, WebIO

### Compartments
- Sample A: T1 = 100ms, T2 = 50ms 
- Sample B: T1 = 50ms, T2 = 25ms

### Scanner

- B0 =     1.5 [T] - main magnetic field
- B1 =    1.0e-5 [T] - maximum RF amplitude
- Gmax =   0.06 [T/m] - maximum gradient
- Smax =   500 [mT/m/ms] - maximum slew-rate
- ADC_Δt = 2.0e-6 [s] - ADC raster time
- seq_Δt = 1.0e-5 [s] - sequence-block raster time
- GR_Δt =  1.0e-5 [s] - gradient raster time
- RF_Δt =  1.0e-6 [s] - RF raster time
- RF_ring_down_T = 2.0e-5 [s] - RF ring down time
- RF_dead_time_T = 0.0001 [s] - RF dead time
- ADC_dead_time_T = 1.0e-5 [s] - ADC dead time

### MATLAB data

In [ ]:
function read_data(file_path)
    # MATLAB data
    mat_data = matread(file_path)
    RF_Hz = mat_data["b1"]
    tf_sp = mat_data["tf_s"]
    t_sp = mat_data["t_s"]
    Mmax = mat_data["Mmax"]
    Mmin = mat_data["Mmin"]

    return RF_Hz, tf_sp, t_sp, Mmax, Mmin
end

### Functions

#### Sequence

In [ ]:
function create_sequence(RF_T, tf_sp, nADC, durADC)
    # RF block
    exc = RF(RF_T', tf_sp)
    
    # ADC block
    aqc = ADC(nADC, durADC)
    
    # Concatenating
    seq = Sequence()
    seq += exc
    seq += aqc
    
    return seq
end

#### Signal

In [ ]:
function simulate_signal(obj, seq, sys)
    signal = simulate(obj, seq, sys; simParams=Dict("return_type" => "state"))
    Mx = real(signal.xy)[:]
    My = imag(signal.xy)[:]
    Mz = signal.z[:]

    return Mx, My, Mz
end

#### Fidelity

In [ ]:
function calculate_fidelity(Mx, My, Mz, Mref)
    fidelity_Mx = round(abs(Mx[] - Mref[2, end])*100, digits=2)
    fidelity_My = round(abs(My[] - Mref[3, end])*100, digits=2)
    fidelity_Mz = round(abs(Mz[] - Mref[4, end])*100, digits=2)

    return fidelity_Mx, fidelity_My, fidelity_Mz
end

#### Magnetization Dynamics

In [ ]:
function simulate_magnetization_dynamics(RF_T, t_sp, sys, obj1, nADC, durADC)
    pieces = length(RF_T)
    M_koma = zeros(Float64, 3, pieces)
    t_koma = zeros(Float64, 1, pieces)
    
    for i in 1:pieces
        blocks = Int(length(RF_T) / pieces)
        rf_block = RF_T[1, 1:i*blocks]'
        t_block = t_sp[1, i*blocks]

        seq1 = create_sequence(rf_block, t_block, nADC, durADC)

        Mx, My, Mz = simulate_signal(obj1, seq1, sys)

        M_koma[1, i] = Mx[]
        M_koma[2, i] = My[]
        M_koma[3, i] = Mz[]
        t_koma[1, i] = t_block
    end
    
    return M_koma[2, :], M_koma[3, :], t_koma'
end

#### Plots

In [ ]:
function plot_results(t_evol, Mmax, t_koma, Mz_koma)
    plotly()
    plot(t_evol, Mmax[4, :], line=:solid, marker=:circle, label="OC Grape")
    plot!(t_koma, Mz_koma, seriestype=:line, marker=:, label="KomaMRI")
end

### Data for simulations

#### Load MATLAB data

In [ ]:
oc_path = "/Users/amandanicotina/Documents/Julia/Projects/KomaMRIScripts/OC_Grape/OC_fields/oc_field.mat"
RF_Hz, tf_sp, t_sp, Mmax, Mmin = read_data(oc_path)

#### Scanner

In [ ]:
sys = Scanner()

#### Phantom

In [ ]:
obj = 
Phantom{Float64}(name = "spin1", x = [0.], T1 = [274e-3], T2 = [237e-3], Δw = [2π*7450]);
Mref = Mmax;

#### Create sequence

In [ ]:
# convert to Tesla
RF_T = RF_Hz/γ;

# 1st block -> RF block
exc = RF(RF_T', tf_sp);

# 2nd block -> ADC block
nADC = 1 ;
durADC = 1e-3 ;

seq =  create_sequence(RF_T, tf_sp, nADC, durADC);

# Plot Sequence
# p1 = plot_seq(seq; slider = false, height = 300, max_rf_samples=Inf)

#### Calculate signal and fidelity

In [ ]:
(Mx, My, Mz) = simulate_signal(obj, seq, sys)
(Fx, Fy, Fz) = calculate_fidelity(Mx, My, Mz, Mref)

#### Simulate magnetization dynamics

In [ ]:
(Mykoma, Mzkoma, tkoma) = 
simulate_magnetization_dynamics(RF_T, t_sp, sys, obj, nADC, durADC)

#### Plots

In [ ]:
plot_results(t_evol, Mmax, tkoma, Mzkoma)

### Check x and z which is which
